In [1]:
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import string
from ipydatagrid import DataGrid, TextRenderer, VegaExpr
import csv
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
today = datetime.today().strftime("%Y/%m/%d")
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.integrate import simpson
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numbers

Data Processing Workflow

In [2]:
def create_table(num_rows, num_cols, row_labels=None, col_labels=None):
    """
    Creates a pandas DataFrame with specified dimensions and labels.
    """
    if row_labels is None:
        row_labels = list(string.ascii_uppercase[:num_rows])
    if col_labels is None:
        col_labels = range(1, num_cols+1)
    # The data is cast to float to ensure it's a numeric type for the mean calculation
    data = [[(i+1)*(j+1) for j in range(num_cols)] for i in range(num_rows)]
    return pd.DataFrame(data, index=row_labels, columns=col_labels, dtype=object)


    
    
def extract_plate_data(filepath= 'AT5 hill plots.csv'):

    # Initialize a dictionary to store the extracted data.
    data_dict = defaultdict(list)
    current_key = None
    key_counts = defaultdict(int)  # Track duplicate key counts

    # Read the CSV file line by line.
    with open(filepath, 'r', encoding='latin1') as file:
        reader = csv.reader(file)
        for row in reader:
            if row:
                first_cell = row[0].strip()
                

                # Logic for when the first cell contains a colon.
                if ':' in first_cell and first_cell.count(':') == 1:
                    key, value_part = first_cell.split(':', 1)
                    key = key.strip()
                    # Handle duplicate keys
                    if key in data_dict:
                        key_counts[key] += 1
                        key = f"{key}_{key_counts[key]}"
                    current_key = key
                    values = [value_part.strip()] + [cell.strip() for cell in row[1:] if cell.strip()]
                    if values:
                        data_dict[current_key].extend(values)

                # Logic for when the first cell is not blank and does not contain a colon.
                elif first_cell:
                    key = first_cell
                    # Handle duplicate keys
                    if key in data_dict:
                        key_counts[key] += 1
                        key = f"{key}_{key_counts[key]}"
                    current_key = key
                    values = [cell.strip() for cell in row[1:] if cell.strip()]
                    if values:
                        data_dict[current_key].extend(values)

                # Logic for when the first cell is blank, the rest of the row belongs to the current key.
                elif current_key:
                    values = [cell.strip() for cell in row[1:] if cell.strip()]
                    if values:
                        data_dict[current_key].extend(values)
    return data_dict



# Function to extract a single element (0=compound, 1=lipid, 2=conc) from the dictionary
def dict_to_plate_df(layout_dict, element_index):
    rows = ['A','B','C','D','E','F','G','H']
    cols = list(range(1,13))
    
    df = pd.DataFrame(index=rows, columns=cols)
    
    for well, value in layout_dict.items():
        row = well[0]
        col = int(well[1:])
        df.loc[row, col] = value[element_index]  # extract compound/lipid/conc
    
    return df

def wells_with_value(layout_dict, target_value):
    wells = [well for well, value in layout_dict.items() if target_value in value]
    return wells

In [ ]:
# --- Concentration Sets Used ---
C_HIGH = [30, 20, 13.333333, 8.888889, 5.925926, 3.950617, 2.633745] # Compound 1 set (Variable) / Uniform set (Short Format)
C_LOW = [10, 5, 2.5, 1.25, 0.625, 0.3125, 0.15625]                   # Compound 2 set (Variable)
C_MID = [15, 7.5, 3.75, 1.875, 0.9375, 0.46875, 0.234375]            # Compound 3 set (Variable)

# Helper function to get the concentration index based on row letter
ROW_MAP = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}

# --- FINALIZED PLATE LAYOUT 1: Short_Format_Layout (Uniform C_HIGH Concentrations) ---
# All compounds use the concentration set: [30, 20, 13.333333, 8.888889, 5.925926, 3.950617, 2.633745]

Short_Format_Layout = {
    'A1': ['Compound 1', 'lipid x', C_HIGH[0]], 'A2': ['Compound 1', 'lipid x', C_HIGH[0]], 'A3': ['Compound 1', 'lipid x', C_HIGH[0]], 'A4': ['Compound 1', 'lipid x', C_HIGH[0]],
    'A5': ['Compound 2', 'lipid x', C_HIGH[0]], 'A6': ['Compound 2', 'lipid x', C_HIGH[0]], 'A7': ['Compound 2', 'lipid x', C_HIGH[0]], 'A8': ['Compound 2', 'lipid x', C_HIGH[0]],
    'A9': ['Compound 3', 'lipid x', C_HIGH[0]], 'A10': ['Compound 3', 'lipid x', C_HIGH[0]], 'A11': ['Compound 3', 'lipid x', C_HIGH[0]], 'A12': ['Compound 3', 'lipid x', C_HIGH[0]],

    'B1': ['Compound 1', 'lipid x', C_HIGH[1]], 'B2': ['Compound 1', 'lipid x', C_HIGH[1]], 'B3': ['Compound 1', 'lipid x', C_HIGH[1]], 'B4': ['Compound 1', 'lipid x', C_HIGH[1]],
    'B5': ['Compound 2', 'lipid x', C_HIGH[1]], 'B6': ['Compound 2', 'lipid x', C_HIGH[1]], 'B7': ['Compound 2', 'lipid x', C_HIGH[1]], 'B8': ['Compound 2', 'lipid x', C_HIGH[1]],
    'B9': ['Compound 3', 'lipid x', C_HIGH[1]], 'B10': ['Compound 3', 'lipid x', C_HIGH[1]], 'B11': ['Compound 3', 'lipid x', C_HIGH[1]], 'B12': ['Compound 3', 'lipid x', C_HIGH[1]],

    'C1': ['Compound 1', 'lipid x', C_HIGH[2]], 'C2': ['Compound 1', 'lipid x', C_HIGH[2]], 'C3': ['Compound 1', 'lipid x', C_HIGH[2]], 'C4': ['Compound 1', 'lipid x', C_HIGH[2]],
    'C5': ['Compound 2', 'lipid x', C_HIGH[2]], 'C6': ['Compound 2', 'lipid x', C_HIGH[2]], 'C7': ['Compound 2', 'lipid x', C_HIGH[2]], 'C8': ['Compound 2', 'lipid x', C_HIGH[2]],
    'C9': ['Compound 3', 'lipid x', C_HIGH[2]], 'C10': ['Compound 3', 'lipid x', C_HIGH[2]], 'C11': ['Compound 3', 'lipid x', C_HIGH[2]], 'C12': ['Compound 3', 'lipid x', C_HIGH[2]],

    'D1': ['Compound 1', 'lipid x', C_HIGH[3]], 'D2': ['Compound 1', 'lipid x', C_HIGH[3]], 'D3': ['Compound 1', 'lipid x', C_HIGH[3]], 'D4': ['Compound 1', 'lipid x', C_HIGH[3]],
    'D5': ['Compound 2', 'lipid x', C_HIGH[3]], 'D6': ['Compound 2', 'lipid x', C_HIGH[3]], 'D7': ['Compound 2', 'lipid x', C_HIGH[3]], 'D8': ['Compound 2', 'lipid x', C_HIGH[3]],
    'D9': ['Compound 3', 'lipid x', C_HIGH[3]], 'D10': ['Compound 3', 'lipid x', C_HIGH[3]], 'D11': ['Compound 3', 'lipid x', C_HIGH[3]], 'D12': ['Compound 3', 'lipid x', C_HIGH[3]],

    'E1': ['Compound 1', 'lipid x', C_HIGH[4]], 'E2': ['Compound 1', 'lipid x', C_HIGH[4]], 'E3': ['Compound 1', 'lipid x', C_HIGH[4]], 'E4': ['Compound 1', 'lipid x', C_HIGH[4]],
    'E5': ['Compound 2', 'lipid x', C_HIGH[4]], 'E6': ['Compound 2', 'lipid x', C_HIGH[4]], 'E7': ['Compound 2', 'lipid x', C_HIGH[4]], 'E8': ['Compound 2', 'lipid x', C_HIGH[4]],
    'E9': ['Compound 3', 'lipid x', C_HIGH[4]], 'E10': ['Compound 3', 'lipid x', C_HIGH[4]], 'E11': ['Compound 3', 'lipid x', C_HIGH[4]], 'E12': ['Compound 3', 'lipid x', C_HIGH[4]],

    'F1': ['Compound 1', 'lipid x', C_HIGH[5]], 'F2': ['Compound 1', 'lipid x', C_HIGH[5]], 'F3': ['Compound 1', 'lipid x', C_HIGH[5]], 'F4': ['Compound 1', 'lipid x', C_HIGH[5]],
    'F5': ['Compound 2', 'lipid x', C_HIGH[5]], 'F6': ['Compound 2', 'lipid x', C_HIGH[5]], 'F7': ['Compound 2', 'lipid x', C_HIGH[5]], 'F8': ['Compound 2', 'lipid x', C_HIGH[5]],
    'F9': ['Compound 3', 'lipid x', C_HIGH[5]], 'F10': ['Compound 3', 'lipid x', C_HIGH[5]], 'F11': ['Compound 3', 'lipid x', C_HIGH[5]], 'F12': ['Compound 3', 'lipid x', C_HIGH[5]],

    'G1': ['Compound 1', 'lipid x', C_HIGH[6]], 'G2': ['Compound 1', 'lipid x', C_HIGH[6]], 'G3': ['Compound 1', 'lipid x', C_HIGH[6]], 'G4': ['Compound 1', 'lipid x', C_HIGH[6]],
    'G5': ['Compound 2', 'lipid x', C_HIGH[6]], 'G6': ['Compound 2', 'lipid x', C_HIGH[6]], 'G7': ['Compound 2', 'lipid x', C_HIGH[6]], 'G8': ['Compound 2', 'lipid x', C_HIGH[6]],
    'G9': ['Compound 3', 'lipid x', C_HIGH[6]], 'G10': ['Compound 3', 'lipid x', C_HIGH[6]], 'G11': ['Compound 3', 'lipid x', C_HIGH[6]], 'G12': ['Compound 3', 'lipid x', C_HIGH[6]],

    'H1': ['DMSO control', '', ''], 'H2': ['DMSO control', '', ''], 'H3': ['DMSO control', '', ''], 'H4': ['DMSO control', '', ''],
    'H5': ['positive control transporter 1', '', ''], 'H6': ['positive control transporter 1', '', ''], 'H7': ['positive control transporter 1', '', ''], 'H8': ['positive control transporter 1', '', ''],
    'H9': ['positive control transporter 1', '', ''], 'H10': ['positive control transporter 1', '', ''], 'H11': ['positive control transporter 1', '', ''], 'H12': ['positive control transporter 1', '', '']
}

Short_Format_Layout_Pippette_error = {
    'A1': ['Compound 1', 'lipid x', C_HIGH[0]], 'A2': ['Compound 1', 'lipid x', C_HIGH[0]], 'A3': ['Compound 1', 'lipid x', C_HIGH[0]], 'A4': ['Compound 1', 'lipid x', C_HIGH[0]],
    'A5': ['Compound 2', 'lipid x', C_HIGH[0]], 'A6': ['Compound 2', 'lipid x', C_HIGH[0]], 'A7': ['Compound 2', 'lipid x', C_HIGH[0]], 'A8': ['Compound 2', 'lipid x', C_HIGH[0]],
    'A9': ['Compound 2', 'lipid x', C_HIGH[0]], 'A10': ['Compound 3', 'lipid x', C_HIGH[0]], 'A11': ['Compound 3', 'lipid x', C_HIGH[0]], 'A12': ['Compound 3', 'lipid x', C_HIGH[0]],

    'B1': ['Compound 1', 'lipid x', C_HIGH[1]], 'B2': ['Compound 1', 'lipid x', C_HIGH[1]], 'B3': ['Compound 1', 'lipid x', C_HIGH[1]], 'B4': ['Compound 1', 'lipid x', C_HIGH[1]],
    'B5': ['Compound 2', 'lipid x', C_HIGH[1]], 'B6': ['Compound 2', 'lipid x', C_HIGH[1]], 'B7': ['Compound 2', 'lipid x', C_HIGH[1]], 'B8': ['Compound 2', 'lipid x', C_HIGH[1]],
    'B9': ['Compound 2', 'lipid x', C_HIGH[1]], 'B10': ['Compound 3', 'lipid x', C_HIGH[1]], 'B11': ['Compound 3', 'lipid x', C_HIGH[1]], 'B12': ['Compound 3', 'lipid x', C_HIGH[1]],

    'C1': ['Compound 1', 'lipid x', C_HIGH[2]], 'C2': ['Compound 1', 'lipid x', C_HIGH[2]], 'C3': ['Compound 1', 'lipid x', C_HIGH[2]], 'C4': ['Compound 1', 'lipid x', C_HIGH[2]],
    'C5': ['Compound 2', 'lipid x', C_HIGH[2]], 'C6': ['Compound 2', 'lipid x', C_HIGH[2]], 'C7': ['Compound 2', 'lipid x', C_HIGH[2]], 'C8': ['Compound 2', 'lipid x', C_HIGH[2]],
    'C9': ['Compound 2', 'lipid x', C_HIGH[2]], 'C10': ['Compound 3', 'lipid x', C_HIGH[2]], 'C11': ['Compound 3', 'lipid x', C_HIGH[2]], 'C12': ['Compound 3', 'lipid x', C_HIGH[2]],

    'D1': ['Compound 1', 'lipid x', C_HIGH[3]], 'D2': ['Compound 1', 'lipid x', C_HIGH[3]], 'D3': ['Compound 1', 'lipid x', C_HIGH[3]], 'D4': ['Compound 1', 'lipid x', C_HIGH[3]],
    'D5': ['Compound 2', 'lipid x', C_HIGH[3]], 'D6': ['Compound 2', 'lipid x', C_HIGH[3]], 'D7': ['Compound 2', 'lipid x', C_HIGH[3]], 'D8': ['Compound 2', 'lipid x', C_HIGH[3]],
    'D9': ['Compound 2', 'lipid x', C_HIGH[3]], 'D10': ['Compound 3', 'lipid x', C_HIGH[3]], 'D11': ['Compound 3', 'lipid x', C_HIGH[3]], 'D12': ['Compound 3', 'lipid x', C_HIGH[3]],

    'E1': ['Compound 1', 'lipid x', C_HIGH[4]], 'E2': ['Compound 1', 'lipid x', C_HIGH[4]], 'E3': ['Compound 1', 'lipid x', C_HIGH[4]], 'E4': ['Compound 1', 'lipid x', C_HIGH[4]],
    'E5': ['Compound 2', 'lipid x', C_HIGH[4]], 'E6': ['Compound 2', 'lipid x', C_HIGH[4]], 'E7': ['Compound 2', 'lipid x', C_HIGH[4]], 'E8': ['Compound 2', 'lipid x', C_HIGH[4]],
    'E9': ['Compound 2', 'lipid x', C_HIGH[4]], 'E10': ['Compound 3', 'lipid x', C_HIGH[4]], 'E11': ['Compound 3', 'lipid x', C_HIGH[4]], 'E12': ['Compound 3', 'lipid x', C_HIGH[4]],

    'F1': ['Compound 1', 'lipid x', C_HIGH[5]], 'F2': ['Compound 1', 'lipid x', C_HIGH[5]], 'F3': ['Compound 1', 'lipid x', C_HIGH[5]], 'F4': ['Compound 1', 'lipid x', C_HIGH[5]],
    'F5': ['Compound 2', 'lipid x', C_HIGH[5]], 'F6': ['Compound 2', 'lipid x', C_HIGH[5]], 'F7': ['Compound 2', 'lipid x', C_HIGH[5]], 'F8': ['Compound 2', 'lipid x', C_HIGH[5]],
    'F9': ['Compound 2', 'lipid x', C_HIGH[5]], 'F10': ['Compound 3', 'lipid x', C_HIGH[5]], 'F11': ['Compound 3', 'lipid x', C_HIGH[5]], 'F12': ['Compound 3', 'lipid x', C_HIGH[5]],

    'G1': ['Compound 1', 'lipid x', C_HIGH[6]], 'G2': ['Compound 1', 'lipid x', C_HIGH[6]], 'G3': ['Compound 1', 'lipid x', C_HIGH[6]], 'G4': ['Compound 1', 'lipid x', C_HIGH[6]],
    'G5': ['Compound 2', 'lipid x', C_HIGH[6]], 'G6': ['Compound 2', 'lipid x', C_HIGH[6]], 'G7': ['Compound 2', 'lipid x', C_HIGH[6]], 'G8': ['Compound 2', 'lipid x', C_HIGH[6]],
    'G9': ['Compound 2', 'lipid x', C_HIGH[6]], 'G10': ['Compound 3', 'lipid x', C_HIGH[6]], 'G11': ['Compound 3', 'lipid x', C_HIGH[6]], 'G12': ['Compound 3', 'lipid x', C_HIGH[6]],

    'H1': ['DMSO control', '', ''], 'H2': ['DMSO control', '', ''], 'H3': ['DMSO control', '', ''], 'H4': ['DMSO control', '', ''],
    'H5': ['positive control transporter 1', '', ''], 'H6': ['positive control transporter 1', '', ''], 'H7': ['positive control transporter 1', '', ''], 'H8': ['positive control transporter 1', '', ''],
    'H9': ['positive control transporter 1', '', ''], 'H10': ['positive control transporter 1', '', ''], 'H11': ['positive control transporter 1', '', ''], 'H12': ['positive control transporter 1', '', '']
}

# --- FINALIZED PLATE LAYOUT 2: Short_Format_Layout_variable_concs (Compound-Specific Concentrations) ---
# Compound 1: C_HIGH (30 down to 2.633745)
# Compound 2: C_LOW (10 down to 0.15625)
# Compound 3: C_MID (15 down to 0.234375)

Short_Format_Layout_variable_concs = {
    # Row A (Index 0)
    'A1': ['Compound 1', 'lipid x', C_HIGH[0]], 'A2': ['Compound 1', 'lipid x', C_HIGH[0]], 'A3': ['Compound 1', 'lipid x', C_HIGH[0]], 'A4': ['Compound 1', 'lipid x', C_HIGH[0]],
    'A5': ['Compound 2', 'lipid x', C_LOW[0]], 'A6': ['Compound 2', 'lipid x', C_LOW[0]], 'A7': ['Compound 2', 'lipid x', C_LOW[0]], 'A8': ['Compound 2', 'lipid x', C_LOW[0]],
    'A9': ['Compound 3', 'lipid x', C_MID[0]], 'A10': ['Compound 3', 'lipid x', C_MID[0]], 'A11': ['Compound 3', 'lipid x', C_MID[0]], 'A12': ['Compound 3', 'lipid x', C_MID[0]],

    # Row B (Index 1)
    'B1': ['Compound 1', 'lipid x', C_HIGH[1]], 'B2': ['Compound 1', 'lipid x', C_HIGH[1]], 'B3': ['Compound 1', 'lipid x', C_HIGH[1]], 'B4': ['Compound 1', 'lipid x', C_HIGH[1]],
    'B5': ['Compound 2', 'lipid x', C_LOW[1]], 'B6': ['Compound 2', 'lipid x', C_LOW[1]], 'B7': ['Compound 2', 'lipid x', C_LOW[1]], 'B8': ['Compound 2', 'lipid x', C_LOW[1]],
    'B9': ['Compound 3', 'lipid x', C_MID[1]], 'B10': ['Compound 3', 'lipid x', C_MID[1]], 'B11': ['Compound 3', 'lipid x', C_MID[1]], 'B12': ['Compound 3', 'lipid x', C_MID[1]],

    # Row C (Index 2)
    'C1': ['Compound 1', 'lipid x', C_HIGH[2]], 'C2': ['Compound 1', 'lipid x', C_HIGH[2]], 'C3': ['Compound 1', 'lipid x', C_HIGH[2]], 'C4': ['Compound 1', 'lipid x', C_HIGH[2]],
    'C5': ['Compound 2', 'lipid x', C_LOW[2]], 'C6': ['Compound 2', 'lipid x', C_LOW[2]], 'C7': ['Compound 2', 'lipid x', C_LOW[2]], 'C8': ['Compound 2', 'lipid x', C_LOW[2]],
    'C9': ['Compound 3', 'lipid x', C_MID[2]], 'C10': ['Compound 3', 'lipid x', C_MID[2]], 'C11': ['Compound 3', 'lipid x', C_MID[2]], 'C12': ['Compound 3', 'lipid x', C_MID[2]],

    # Row D (Index 3)
    'D1': ['Compound 1', 'lipid x', C_HIGH[3]], 'D2': ['Compound 1', 'lipid x', C_HIGH[3]], 'D3': ['Compound 1', 'lipid x', C_HIGH[3]], 'D4': ['Compound 1', 'lipid x', C_HIGH[3]],
    'D5': ['Compound 2', 'lipid x', C_LOW[3]], 'D6': ['Compound 2', 'lipid x', C_LOW[3]], 'D7': ['Compound 2', 'lipid x', C_LOW[3]], 'D8': ['Compound 2', 'lipid x', C_LOW[3]],
    'D9': ['Compound 3', 'lipid x', C_MID[3]], 'D10': ['Compound 3', 'lipid x', C_MID[3]], 'D11': ['Compound 3', 'lipid x', C_MID[3]], 'D12': ['Compound 3', 'lipid x', C_MID[3]],

    # Row E (Index 4)
    'E1': ['Compound 1', 'lipid x', C_HIGH[4]], 'E2': ['Compound 1', 'lipid x', C_HIGH[4]], 'E3': ['Compound 1', 'lipid x', C_HIGH[4]], 'E4': ['Compound 1', 'lipid x', C_HIGH[4]],
    'E5': ['Compound 2', 'lipid x', C_LOW[4]], 'E6': ['Compound 2', 'lipid x', C_LOW[4]], 'E7': ['Compound 2', 'lipid x', C_LOW[4]], 'E8': ['Compound 2', 'lipid x', C_LOW[4]],
    'E9': ['Compound 3', 'lipid x', C_MID[4]], 'E10': ['Compound 3', 'lipid x', C_MID[4]], 'E11': ['Compound 3', 'lipid x', C_MID[4]], 'E12': ['Compound 3', 'lipid x', C_MID[4]],

    # Row F (Index 5)
    'F1': ['Compound 1', 'lipid x', C_HIGH[5]], 'F2': ['Compound 1', 'lipid x', C_HIGH[5]], 'F3': ['Compound 1', 'lipid x', C_HIGH[5]], 'F4': ['Compound 1', 'lipid x', C_HIGH[5]],
    'F5': ['Compound 2', 'lipid x', C_LOW[5]], 'F6': ['Compound 2', 'lipid x', C_LOW[5]], 'F7': ['Compound 2', 'lipid x', C_LOW[5]], 'F8': ['Compound 2', 'lipid x', C_LOW[5]],
    'F9': ['Compound 3', 'lipid x', C_MID[5]], 'F10': ['Compound 3', 'lipid x', C_MID[5]], 'F11': ['Compound 3', 'lipid x', C_MID[5]], 'F12': ['Compound 3', 'lipid x', C_MID[5]],

    # Row G (Index 6)
    'G1': ['Compound 1', 'lipid x', C_HIGH[6]], 'G2': ['Compound 1', 'lipid x', C_HIGH[6]], 'G3': ['Compound 1', 'lipid x', C_HIGH[6]], 'G4': ['Compound 1', 'lipid x', C_HIGH[6]],
    'G5': ['Compound 2', 'lipid x', C_LOW[6]], 'G6': ['Compound 2', 'lipid x', C_LOW[6]], 'G7': ['Compound 2', 'lipid x', C_LOW[6]], 'G8': ['Compound 2', 'lipid x', C_LOW[6]],
    'G9': ['Compound 3', 'lipid x', C_MID[6]], 'G10': ['Compound 3', 'lipid x', C_MID[6]], 'G11': ['Compound 3', 'lipid x', C_MID[6]], 'G12': ['Compound 3', 'lipid x', C_MID[6]],

    # Row H (Controls)
    'H1': ['DMSO control', '', ''], 'H2': ['DMSO control', '', ''], 'H3': ['DMSO control', '', ''], 'H4': ['DMSO control', '', ''],
    'H5': ['positive control transporter 1', '', ''], 'H6': ['positive control transporter 1', '', ''], 'H7': ['positive control transporter 1', '', ''], 'H8': ['positive control transporter 1', '', ''],
    'H9': ['positive control transporter 1', '', ''], 'H10': ['positive control transporter 1', '', ''], 'H11': ['positive control transporter 1', '', ''], 'H12': ['positive control transporter 1', '', '']
}

Screening = {
    'A1': ['Compound 1', 'lipid x', 'conc 1'], 'A2': ['Compound 2', 'lipid x', 'conc 1'], 'A3': ['Compound 3', 'lipid x', 'conc 1'], 'A4': ['Compound 4', 'lipid x', 'conc 1'],
    'A5': ['Compound 5', 'lipid x', 'conc 1'], 'A6': ['Compound 6', 'lipid x', 'conc 1'], 'A7': ['Compound 7', 'lipid x', 'conc 1'], 'A8': ['Compound 8', 'lipid x', 'conc 1'],
    'A9': ['Compound 9', 'lipid x', 'conc 1'], 'A10': ['Compound 10', 'lipid x', 'conc 1'], 'A11': ['Compound 11', 'lipid x', 'conc 1'], 'A12': ['Compound 12', 'lipid x', 'conc 1'],

    'B1': ['Compound 13', 'lipid x', 'conc 1'], 'B2': ['Compound 14', 'lipid x', 'conc 1'], 'B3': ['Compound 15', 'lipid x', 'conc 1'], 'B4': ['Compound 16', 'lipid x', 'conc 1'],
    'B5': ['Compound 17', 'lipid x', 'conc 1'], 'B6': ['Compound 18', 'lipid x', 'conc 1'], 'B7': ['Compound 19', 'lipid x', 'conc 1'], 'B8': ['Compound 20', 'lipid x', 'conc 1'],
    'B9': ['Compound 21', 'lipid x', 'conc 1'], 'B10': ['Compound 22', 'lipid x', 'conc 1'], 'B11': ['Compound 23', 'lipid x', 'conc 1'], 'B12': ['Compound 24', 'lipid x', 'conc 1'],

    'C1': ['Compound 25', 'lipid x', 'conc 1'], 'C2': ['Compound 26', 'lipid x', 'conc 1'], 'C3': ['Compound 27', 'lipid x', 'conc 1'], 'C4': ['Compound 28', 'lipid x', 'conc 1'],
    'C5': ['Compound 29', 'lipid x', 'conc 1'], 'C6': ['Compound 30', 'lipid x', 'conc 1'], 'C7': ['Compound 31', 'lipid x', 'conc 1'], 'C8': ['Compound 32', 'lipid x', 'conc 1'],
    'C9': ['Compound 33', 'lipid x', 'conc 1'], 'C10': ['Compound 34', 'lipid x', 'conc 1'], 'C11': ['Compound 35', 'lipid x', 'conc 1'], 'C12': ['Compound 36', 'lipid x', 'conc 1'],

    'D1': ['Compound 37', 'lipid x', 'conc 1'], 'D2': ['Compound 38', 'lipid x', 'conc 1'], 'D3': ['Compound 39', 'lipid x', 'conc 1'], 'D4': ['Compound 40', 'lipid x', 'conc 1'],
    'D5': ['Compound 41', 'lipid x', 'conc 1'], 'D6': ['Compound 42', 'lipid x', 'conc 1'], 'D7': ['Compound 43', 'lipid x', 'conc 1'], 'D8': ['Compound 44', 'lipid x', 'conc 1'],
    'D9': ['Compound 45', 'lipid x', 'conc 1'], 'D10': ['Compound 46', 'lipid x', 'conc 1'], 'D11': ['Compound 47', 'lipid x', 'conc 1'], 'D12': ['Compound 48', 'lipid x', 'conc 1'],

    'E1': ['Compound 49', 'lipid x', 'conc 1'], 'E2': ['Compound 50', 'lipid x', 'conc 1'], 'E3': ['Compound 51', 'lipid x', 'conc 1'], 'E4': ['Compound 52', 'lipid x', 'conc 1'],
    'E5': ['Compound 53', 'lipid x', 'conc 1'], 'E6': ['Compound 54', 'lipid x', 'conc 1'], 'E7': ['Compound 55', 'lipid x', 'conc 1'], 'E8': ['Compound 56', 'lipid x', 'conc 1'],
    'E9': ['Compound 57', 'lipid x', 'conc 1'], 'E10': ['Compound 58', 'lipid x', 'conc 1'], 'E11': ['Compound 59', 'lipid x', 'conc 1'], 'E12': ['Compound 60', 'lipid x', 'conc 1'],

    'F1': ['Compound 61', 'lipid x', 'conc 1'], 'F2': ['Compound 62', 'lipid x', 'conc 1'], 'F3': ['Compound 63', 'lipid x', 'conc 1'], 'F4': ['Compound 64', 'lipid x', 'conc 1'],
    'F5': ['Compound 65', 'lipid x', 'conc 1'], 'F6': ['Compound 66', 'lipid x', 'conc 1'], 'F7': ['Compound 67', 'lipid x', 'conc 1'], 'F8': ['Compound 68', 'lipid x', 'conc 1'],
    'F9': ['Compound 69', 'lipid x', 'conc 1'], 'F10': ['Compound 70', 'lipid x', 'conc 1'], 'F11': ['Compound 71', 'lipid x', 'conc 1'], 'F12': ['Compound 72', 'lipid x', 'conc 1'],

    'G1': ['Compound 73', 'lipid x', 'conc 1'], 'G2': ['Compound 74', 'lipid x', 'conc 1'], 'G3': ['Compound 75', 'lipid x', 'conc 1'], 'G4': ['Compound 76', 'lipid x', 'conc 1'],
    'G5': ['Compound 77', 'lipid x', 'conc 1'], 'G6': ['Compound 78', 'lipid x', 'conc 1'], 'G7': ['Compound 79', 'lipid x', 'conc 1'], 'G8': ['Compound 80', 'lipid x', 'conc 1'],
    'G9': ['Compound 81', 'lipid x', 'conc 1'], 'G10': ['Compound 82', 'lipid x', 'conc 1'], 'G11': ['Compound 83', 'lipid x', 'conc 1'], 'G12': ['Compound 84', 'lipid x', 'conc 1'],

    'H1': ['DMSO control', '', ''], 'H2': ['DMSO control', '', ''], 'H3': ['DMSO control', '', ''], 'H4': ['DMSO control', '', ''],
    'H5': ['positive control transporter 1', '', ''], 'H6': ['positive control transporter 1', '', ''], 'H7': ['positive control transporter 1', '', ''], 'H8': ['positive control transporter 1', '', ''],
    'H9': ['positive control 2 or triton', '', ''], 'H10': ['positive control 2 or triton', '', ''], 'H11': ['positive control 2 or triton', '', ''], 'H12': ['positive control 2 or triton', '', '']
}


Long_Format_Layout = {
    'A1': ['Compound 1', 'lipid x', 'conc 1'], 'A2': ['Compound 1', 'lipid x', 'conc 2'], 'A3': ['Compound 1', 'lipid x', 'conc 3'],
    'A4': ['Compound 1', 'lipid x', 'conc 4'], 'A5': ['Compound 1', 'lipid x', 'conc 5'], 'A6': ['Compound 1', 'lipid x', 'conc 6'],
    'A7': ['Compound 1', 'lipid x', 'conc 7'], 'A8': ['Compound 1', 'lipid x', 'conc 8'], 'A9': ['Compound 1', 'lipid x', 'conc 9'],
    'A10': ['Compound 1', 'lipid x', 'conc 10'], 'A11': ['Compound 1', 'lipid x', 'conc 11'],
    'B1': ['Compound 1', 'lipid x', 'conc 1'], 'B2': ['Compound 1', 'lipid x', 'conc 2'], 'B3': ['Compound 1', 'lipid x', 'conc 3'],
    'B4': ['Compound 1', 'lipid x', 'conc 4'], 'B5': ['Compound 1', 'lipid x', 'conc 5'], 'B6': ['Compound 1', 'lipid x', 'conc 6'],
    'B7': ['Compound 1', 'lipid x', 'conc 7'], 'B8': ['Compound 1', 'lipid x', 'conc 8'], 'B9': ['Compound 1', 'lipid x', 'conc 9'],
    'B10': ['Compound 1', 'lipid x', 'conc 10'], 'B11': ['Compound 1', 'lipid x', 'conc 11'],
    'C1': ['Compound 1', 'lipid x', 'conc 1'], 'C2': ['Compound 1', 'lipid x', 'conc 2'], 'C3': ['Compound 1', 'lipid x', 'conc 3'],
    'C4': ['Compound 1', 'lipid x', 'conc 4'], 'C5': ['Compound 1', 'lipid x', 'conc 5'], 'C6': ['Compound 1', 'lipid x', 'conc 6'],
    'C7': ['Compound 1', 'lipid x', 'conc 7'], 'C8': ['Compound 1', 'lipid x', 'conc 8'], 'C9': ['Compound 1', 'lipid x', 'conc 9'],
    'C10': ['Compound 1', 'lipid x', 'conc 10'], 'C11': ['Compound 1', 'lipid x', 'conc 11'],
    'D1': ['Compound 1', 'lipid x', 'conc 1'], 'D2': ['Compound 1', 'lipid x', 'conc 2'], 'D3': ['Compound 1', 'lipid x', 'conc 3'],
    'D4': ['Compound 1', 'lipid x', 'conc 4'], 'D5': ['Compound 1', 'lipid x', 'conc 5'], 'D6': ['Compound 1', 'lipid x', 'conc 6'],
    'D7': ['Compound 1', 'lipid x', 'conc 7'], 'D8': ['Compound 1', 'lipid x', 'conc 8'], 'D9': ['Compound 1', 'lipid x', 'conc 9'],
    'D10': ['Compound 1', 'lipid x', 'conc 10'], 'D11': ['Compound 1', 'lipid x', 'conc 11'],
    
    'E1': ['Compound 2', 'lipid x', 'conc 1'], 'E2': ['Compound 2', 'lipid x', 'conc 2'], 'E3': ['Compound 2', 'lipid x', 'conc 3'],
    'E4': ['Compound 2', 'lipid x', 'conc 4'], 'E5': ['Compound 2', 'lipid x', 'conc 5'], 'E6': ['Compound 2', 'lipid x', 'conc 6'],
    'E7': ['Compound 2', 'lipid x', 'conc 7'], 'E8': ['Compound 2', 'lipid x', 'conc 8'], 'E9': ['Compound 2', 'lipid x', 'conc 9'],
    'E10': ['Compound 2', 'lipid x', 'conc 10'], 'E11': ['Compound 2', 'lipid x', 'conc 11'],
    'F1': ['Compound 2', 'lipid x', 'conc 1'], 'F2': ['Compound 2', 'lipid x', 'conc 2'], 'F3': ['Compound 2', 'lipid x', 'conc 3'],
    'F4': ['Compound 2', 'lipid x', 'conc 4'], 'F5': ['Compound 2', 'lipid x', 'conc 5'], 'F6': ['Compound 2', 'lipid x', 'conc 6'],
    'F7': ['Compound 2', 'lipid x', 'conc 7'], 'F8': ['Compound 2', 'lipid x', 'conc 8'], 'F9': ['Compound 2', 'lipid x', 'conc 9'],
    'F10': ['Compound 2', 'lipid x', 'conc 10'], 'F11': ['Compound 2', 'lipid x', 'conc 11'],
    'G1': ['Compound 2', 'lipid x', 'conc 1'], 'G2': ['Compound 2', 'lipid x', 'conc 2'], 'G3': ['Compound 2', 'lipid x', 'conc 3'],
    'G4': ['Compound 2', 'lipid x', 'conc 4'], 'G5': ['Compound 2', 'lipid x', 'conc 5'], 'G6': ['Compound 2', 'lipid x', 'conc 6'],
    'G7': ['Compound 2', 'lipid x', 'conc 7'], 'G8': ['Compound 2', 'lipid x', 'conc 8'], 'G9': ['Compound 2', 'lipid x', 'conc 9'],
    'G10': ['Compound 2', 'lipid x', 'conc 10'], 'G11': ['Compound 2', 'lipid x', 'conc 11'],
    'H1': ['Compound 2', 'lipid x', 'conc 1'], 'H2': ['Compound 2', 'lipid x', 'conc 2'], 'H3': ['Compound 2', 'lipid x', 'conc 3'],
    'H4': ['Compound 2', 'lipid x', 'conc 4'], 'H5': ['Compound 2', 'lipid x', 'conc 5'], 'H6': ['Compound 2', 'lipid x', 'conc 6'],
    'H7': ['Compound 2', 'lipid x', 'conc 7'], 'H8': ['Compound 2', 'lipid x', 'conc 8'], 'H9': ['Compound 2', 'lipid x', 'conc 9'],
    'H10': ['Compound 2', 'lipid x', 'conc 10'], 'H11': ['Compound 2', 'lipid x', 'conc 11'],
    
    'A12': ['DMSO control', '', ''], 'B12': ['DMSO control', '', ''], 'C12': ['DMSO control', '', ''], 'D12': ['DMSO control', '', ''],
    'E12': ['positive control transporter 1', '', ''], 'F12': ['positive control transporter 1', '', ''], 'G12': ['positive control transporter 1', '', ''],
    'H12': ['positive control 2 or triton', '', '']
}

Super_Long_Format_Layout = {
    'A1': ['Compound 1', 'lipid x', 'conc 1'], 'A2': ['Compound 1', 'lipid x', 'conc 2'], 'A3': ['Compound 1', 'lipid x', 'conc 3'],
    'A4': ['Compound 1', 'lipid x', 'conc 4'], 'A5': ['Compound 1', 'lipid x', 'conc 5'], 'A6': ['Compound 1', 'lipid x', 'conc 6'],
    'A7': ['Compound 1', 'lipid x', 'conc 7'], 'A8': ['Compound 1', 'lipid x', 'conc 8'], 'A9': ['Compound 1', 'lipid x', 'conc 9'],
    'A10': ['Compound 1', 'lipid x', 'conc 10'], 'A11': ['Compound 1', 'lipid x', 'conc 11'],
    'B1': ['Compound 1', 'lipid x', 'conc 1'], 'B2': ['Compound 1', 'lipid x', 'conc 2'], 'B3': ['Compound 1', 'lipid x', 'conc 3'],
    'B4': ['Compound 1', 'lipid x', 'conc 4'], 'B5': ['Compound 1', 'lipid x', 'conc 5'], 'B6': ['Compound 1', 'lipid x', 'conc 6'],
    'B7': ['Compound 1', 'lipid x', 'conc 7'], 'B8': ['Compound 1', 'lipid x', 'conc 8'], 'B9': ['Compound 1', 'lipid x', 'conc 9'],
    'B10': ['Compound 1', 'lipid x', 'conc 10'], 'B11': ['Compound 1', 'lipid x', 'conc 11'],
    'C1': ['Compound 1', 'lipid x', 'conc 1'], 'C2': ['Compound 1', 'lipid x', 'conc 2'], 'C3': ['Compound 1', 'lipid x', 'conc 3'],
    'C4': ['Compound 1', 'lipid x', 'conc 4'], 'C5': ['Compound 1', 'lipid x', 'conc 5'], 'C6': ['Compound 1', 'lipid x', 'conc 6'],
    'C7': ['Compound 1', 'lipid x', 'conc 7'], 'C8': ['Compound 1', 'lipid x', 'conc 8'], 'C9': ['Compound 1', 'lipid x', 'conc 9'],
    'C10': ['Compound 1', 'lipid x', 'conc 10'], 'C11': ['Compound 1', 'lipid x', 'conc 11'],
    'D1': ['Compound 1', 'lipid x', 'conc 1'], 'D2': ['Compound 1', 'lipid x', 'conc 2'], 'D3': ['Compound 1', 'lipid x', 'conc 3'],
    'D4': ['Compound 1', 'lipid x', 'conc 4'], 'D5': ['Compound 1', 'lipid x', 'conc 5'], 'D6': ['Compound 1', 'lipid x', 'conc 6'],
    'D7': ['Compound 1', 'lipid x', 'conc 7'], 'D8': ['Compound 1', 'lipid x', 'conc 8'], 'D9': ['Compound 1', 'lipid x', 'conc 9'],
    'D10': ['Compound 1', 'lipid x', 'conc 10'], 'D11': ['Compound 1', 'lipid x', 'conc 11'],
    
    'E1': ['Compound 1', 'lipid x', 'conc 12'], 'E2': ['Compound 1', 'lipid x', 'conc 13'], 'E3': ['Compound 1', 'lipid x', 'conc 14'],
    'E4': ['Compound 1', 'lipid x', 'conc 15'], 'E5': ['Compound 1', 'lipid x', 'conc 16'], 'E6': ['Compound 1', 'lipid x', 'conc 17'],
    'E7': ['Compound 1', 'lipid x', 'conc 18'], 'E8': ['Compound 1', 'lipid x', 'conc 19'], 'E9': ['Compound 1', 'lipid x', 'conc 20'],
    'E10': ['Compound 1', 'lipid x', 'conc 21'], 'E11': ['Compound 1', 'lipid x', 'conc 22'],
    'F1': ['Compound 1', 'lipid x', 'conc 12'], 'F2': ['Compound 1', 'lipid x', 'conc 13'], 'F3': ['Compound 1', 'lipid x', 'conc 14'],
    'F4': ['Compound 1', 'lipid x', 'conc 15'], 'F5': ['Compound 1', 'lipid x', 'conc 16'], 'F6': ['Compound 1', 'lipid x', 'conc 17'],
    'F7': ['Compound 1', 'lipid x', 'conc 18'], 'F8': ['Compound 1', 'lipid x', 'conc 19'], 'F9': ['Compound 1', 'lipid x', 'conc 20'],
    'F10': ['Compound 1', 'lipid x', 'conc 21'], 'F11': ['Compound 1', 'lipid x', 'conc 22'],
    'G1': ['Compound 1', 'lipid x', 'conc 12'], 'G2': ['Compound 1', 'lipid x', 'conc 13'], 'G3': ['Compound 1', 'lipid x', 'conc 14'],
    'G4': ['Compound 1', 'lipid x', 'conc 15'], 'G5': ['Compound 1', 'lipid x', 'conc 16'], 'G6': ['Compound 1', 'lipid x', 'conc 17'],
    'G7': ['Compound 1', 'lipid x', 'conc 18'], 'G8': ['Compound 1', 'lipid x', 'conc 19'], 'G9': ['Compound 1', 'lipid x', 'conc 20'],
    'G10': ['Compound 1', 'lipid x', 'conc 21'], 'G11': ['Compound 1', 'lipid x', 'conc 22'],
    'H1': ['Compound 1', 'lipid x', 'conc 12'], 'H2': ['Compound 1', 'lipid x', 'conc 13'], 'H3': ['Compound 1', 'lipid x', 'conc 14'],
    'H4': ['Compound 1', 'lipid x', 'conc 15'], 'H5': ['Compound 1', 'lipid x', 'conc 16'], 'H6': ['Compound 1', 'lipid x', 'conc 17'],
    'H7': ['Compound 1', 'lipid x', 'conc 18'], 'H8': ['Compound 1', 'lipid x', 'conc 19'], 'H9': ['Compound 1', 'lipid x', 'conc 20'],
    'H10': ['Compound 1', 'lipid x', 'conc 21'], 'H11': ['Compound 1', 'lipid x', 'conc 22'],

    'A12': ['DMSO control', '', ''], 'B12': ['DMSO control', '', ''], 'C12': ['DMSO control', '', ''], 'D12': ['DMSO control', '', ''],
    'E12': ['positive control transporter 1', '', ''], 'F12': ['positive control transporter 1', '', ''], 'G12': ['positive control transporter 1', '', ''],
    'H12': ['positive control 2 or triton', '', '']
}


Format_dict = {'Short_Format_Layout':Short_Format_Layout,'Short_Format_Layout_variable_concs':Short_Format_Layout_variable_concs,'Short_Format_Layout_Pipette_error':Short_Format_Layout_Pippette_error,'Super_Long_Format_Layout':Super_Long_Format_Layout, 'Long_Format_Layout':Long_Format_Layout, 'Screening':Screening}

# Function to get unique values at a specific index
def get_unique_at_index(layout_dict, idx, ordered=True):
    values = {v[idx] for v in layout_dict.values() if len(v) > idx and v[idx] != ''}
    if ordered and idx == 2:  # special handling for concentrations
        numeric_values = {v for v in values if isinstance(v, numbers.Number)}
        if numeric_values:
                # If numeric, convert the set to a list and sort numerically
                sorted_numerics = sorted(list(numeric_values), reverse=True)
                return [str(v) for v in sorted_numerics]
            # Reverse=True for high-to-low concentration
        # Sort numerically by extracting the number after 'conc '
        return sorted(values, key=lambda x: int(x.split(' ')[1]))
    if ordered and idx == 0:
        values = {v[idx] for v in layout_dict.values() if len(v) > idx and v[idx] != '' and 'Compound' in v[idx]}
        return sorted(values, key=lambda x: int(x.split(' ')[1]))

    
    return list(values)




# Metadata dictionary
metadata_dict = {"plate_layout": "Short_Format_Layout", "file_path": "AT5 hill plots.csv"}

# Outputs
initial_output = widgets.Output()
edit_output = widgets.Output()

def create_limited_textarea(name, items):
    """Create a textarea that only allows up to len(items) lines."""
    textarea = widgets.Textarea(
        value="\n".join(items),
        description=f"{name}:",
        layout=widgets.Layout(width='500px', height=f'{max(50, 25*len(items))}px')
    )
    
    max_lines = len(items)
    
    def on_change(change):
        lines = [x.strip() for x in textarea.value.split("\n") if x.strip()]
        # Limit number of lines
        if len(lines) > max_lines:
            lines = lines[:max_lines]
            textarea.value = "\n".join(lines)
        # Update global metadata
        metadata_dict[name.lower()] = lines
    
    textarea.observe(on_change, names='value')
    return textarea

# Step 1 widgets
plate_layout_options = ['Short_Format_Layout','Short_Format_Layout_Pipette_error','Short_Format_Layout_variable_concs','Super_Long_Format_Layout', 'Long_Format_Layout', 'Screening']

plate_layout_widget = widgets.Dropdown(options=plate_layout_options, value=metadata_dict.get("plate_layout", plate_layout_options[0]), description="Plate Layout:", layout=widgets.Layout(width="400px"))

file_path_widget = widgets.Text(value=metadata_dict["file_path"], description="File Path:", layout=widgets.Layout(width="400px"))
submit_button = widgets.Button(description="Submit", button_style="success", layout=widgets.Layout(width="150px"))
    
# Step 2 editable form
def edit_form(current_metadata):
    global unique_compounds, unique_concentrations, unique_lipids
    #DMSO_control = widgets.Text(value=str(wells_with_value(Format_dict[current_metadata["plate_layout"]], "DMSO control")), description="DMSO control", layout=widgets.Layout(width="400px"))
    #transporter = widgets.Text(value=str(wells_with_value(Format_dict[current_metadata["plate_layout"]], "positive control transporter 1")), description="positive control transporter 1", layout=widgets.Layout(width="400px"))
    file_edit = widgets.Text(value=current_metadata["file_path"], description="File path:", layout=widgets.Layout(width="400px"))
    unique_compounds = get_unique_at_index(Format_dict[current_metadata["plate_layout"]], 0)
    unique_lipids = get_unique_at_index(Format_dict[current_metadata["plate_layout"]], 1)
    unique_concentrations = get_unique_at_index(Format_dict[current_metadata["plate_layout"]], 2)
    
    transporter = create_limited_textarea("positive control transporter 1", wells_with_value(Format_dict[current_metadata["plate_layout"]], "positive control transporter 1"))
    DMSO_control = create_limited_textarea("DMSO_control", wells_with_value(Format_dict[current_metadata["plate_layout"]], "DMSO control"))
    compounds_widget = create_limited_textarea("Compounds", unique_compounds)
    lipids_widget = create_limited_textarea("Lipids", unique_lipids)
    concs_widget = create_limited_textarea("Conc", unique_concentrations)
    
    save_button = widgets.Button(description="Save Changes", button_style="info", layout=widgets.Layout(width="150px"))
    
    def save_changes(btn):
        current_metadata["DMSO_control"] = [x.strip() for x in DMSO_control.value.split("\n") if x.strip()]
        current_metadata["positive control transporter 1"] = [x.strip() for x in transporter.value.split("\n") if x.strip()]
        current_metadata["file_path"] = file_edit.value.strip()

        metadata_dict["compounds"] = [x.strip() for x in compounds_widget.value.split("\n") if x.strip()]
        metadata_dict["lipids"] = [x.strip() for x in lipids_widget.value.split("\n") if x.strip()]
        metadata_dict["concentrations"] = [x.strip() for x in concs_widget.value.split("\n") if x.strip()]
        with edit_output:
            clear_output()
            print("Updated metadata")
    save_button.on_click(None, remove=True)  # Remove any previous handlers
    save_button.on_click(save_changes)        
          
    return widgets.VBox([transporter,DMSO_control, compounds_widget, lipids_widget, concs_widget,file_edit, save_button], layout=widgets.Layout(align_items='center'))


# Step 1 submit function
def first_submit(btn):
    metadata_dict["plate_layout"] = plate_layout_widget.value.strip()
    metadata_dict["file_path"] = file_path_widget.value.strip()
   
    with initial_output:
        clear_output()  # Clear old initial form
    with edit_output:
        clear_output()  # Clear previous editable form
        display(edit_form(metadata_dict))
        

# Remove previous click handlers to prevent duplicates

submit_button.on_click(first_submit)

# Display everything centered
display(widgets.VBox([
    plate_layout_widget,
    file_path_widget,
    submit_button,
    initial_output,
    edit_output
], layout=widgets.Layout(align_items='center', justify_content='center')))


In [7]:



data_dict=extract_plate_data(metadata_dict['file_path'])

def populate_dataframe(well_list: list, data_list: list):
   
    rows = [chr(i) for i in range(ord('A'), ord('H') + 1)]
    columns = [str(i) for i in range(1, 13)]
    df = pd.DataFrame(index=rows, columns=columns, dtype=float)
    if len(well_list) != len(data_list):
        raise ValueError("The well_list and data_list must have the same length.")

    # Iterate through the lists simultaneously
    for well, data in zip(well_list, data_list):
        if not isinstance(well, str) or len(well) < 2 or len(well) > 3:
            print(f"Skipping invalid well name: {well}")
            continue

        data_to_insert = data
        
        # 2. Robust Data Type Handling (Fix for 'OVER' error)
        try:
            # Attempt to convert the data to a float
            data_to_insert = float(data)
        except (ValueError, TypeError):
            # If conversion fails (e.g., data is 'OVER', 'N/A', etc.), replace with NaN
            print(f"Non-numeric data '{data}' found for well '{well}'. Converting to NaN.")
            data_to_insert = np.nan

        try:
            # Parse the well name into row and column
            row = well[0]  # The letter (A-H)
            col = well[1:] # The number (1-12)
            
            # Check if the row and column are valid within the DataFrame
            if row in df.index and col in df.columns:
                # Use .loc to safely update the cell with the processed data
                df.loc[row, col] = data_to_insert
            else:
                print(f"Warning: Well '{well}' is out of DataFrame bounds. Skipping.")
                
        except (KeyError, ValueError, IndexError) as e:
            print(f"Error parsing well ID '{well}': {e}. Skipping.")
           
    
    return df.astype(float)


# Use the function to populate the DataFrame
df_403 = populate_dataframe(data_dict['Wavel.'],data_dict['403'])
df_460 = populate_dataframe(data_dict['Wavel.'],data_dict['460'])
df_sub=df_460/df_403
df_sub

metadata_dict

# Function to compute averages
def control_average(df, controls, key):
    wells = controls[key]  # list of well positions for that control
    values = []
    
    for well in wells:
        row = well[0]      
        col = well[1:]  
        values.append(df.loc[row, col])
    return np.mean(values)

# Example usage
avg_dmso = control_average(df_sub, metadata_dict, 'DMSO_control')

avg_pos = control_average(df_sub, metadata_dict, 'positive control transporter 1')
#avg_pos2 = control_average(df_sub, metadata_dict, 'positive control 2 or triton')

#avg_pos=  (avg_pos1+avg_pos2)/2

df_change=((df_sub-avg_dmso)*100)/(avg_pos-avg_dmso)
df_change = df_change.round(1)
df_change = df_change.astype(str)


# Create zipped mappings
zipped_compounds = dict(zip(unique_compounds, metadata_dict['compounds']))
zipped_lipids = dict(zip(unique_lipids, metadata_dict['lipids']))
zipped_concentrations = dict(zip(unique_concentrations, metadata_dict['concentrations']))

layout =Format_dict[metadata_dict['plate_layout']]


for well, values in layout.items():
    orig_compound, orig_lipid, orig_conc = values
    
    # Replace using zipped mappings if original value exists in the mapping
    new_compound = zipped_compounds.get(orig_compound, orig_compound)
    new_lipid = zipped_lipids.get(orig_lipid, orig_lipid)
    new_conc = zipped_concentrations.get(orig_conc, orig_conc)
    
    layout[well] = [new_compound, new_lipid, new_conc]

# Now Format_dict[metadata_dict['plate_layout']] is updated
edited_dict=Format_dict[metadata_dict['plate_layout']]

# Create three separate DataFrames
num_rows, num_cols = 8, 12
df_original = create_table(num_rows, num_cols)
compound_df = dict_to_plate_df(edited_dict, 0)
lipid_df = dict_to_plate_df(edited_dict, 1)
conc_df = dict_to_plate_df(edited_dict, 2)

tables = {
    "Plate %Change": df_change,
    "Concentration": conc_df,
    "Cell line": lipid_df,
    "Compound": compound_df,
}

def create_mean_renderers(df: pd.DataFrame):
    renderers_dict = {}
    for col in df.columns:
        # Compute mean only for numeric values
        numeric_vals = pd.to_numeric(df[col], errors='coerce')
        column_mean = numeric_vals.mean()
        
        # Conditional coloring: ignore non-numeric cells
        color_expr = VegaExpr(
            f"isNaN(cell.value) ? 'white' : (cell.value > {column_mean-20} ? 'white' : 'salmon')"
        )
        
        column_renderer = TextRenderer(
            background_color=color_expr,
            text_color='black',
            horizontal_alignment='center'
        )
        renderers_dict[col] = column_renderer
    return renderers_dict
def get_column_widths(df, min_width=60, max_width=150, char_width=10, padding=0):
  
    widths = {}
    for col in df.columns:
        # Maximum length between header and any cell in the column
        max_len = max(len(str(col)), df[col].astype(str).map(len).max())
        col_width = max(min_width, max_len * char_width + padding)
        widths[col] = min(col_width, max_width)  # enforce max width
    return widths


# Store references to grids and their DataFrames
grid_dict = {}

sections = [widgets.HTML(
    "<h2 style='font-family:Helvetica, Arial, sans-serif; text-align:center; color:#34495e; font-weight: 300;'>Interactive Data Explorer</h2>"
)]

for name, df in tables.items():

    # Sync changes from grid -> DataFrame
    def on_cell_change(event, df=df):
        row_label, col_label = event["row"], event["column"]
        new_val = event["value"]
        row = df.index.get_loc(row_label)
        col = df.columns.get_loc(col_label)
        df.iat[row, col] = new_val

    if name == 'Plate %Change':
        #mean_renderers = create_mean_renderers(df)
        mean_renderers = {}
    else:
        mean_renderers = {}

    column_widths = get_column_widths(df)

    grid = DataGrid(
        df,
        editable=True,
        selection_mode="cell",
        renderers=mean_renderers,
        column_widths=column_widths,
        align_items='center',
        row_height=25,
        layout=widgets.Layout(
            height=f'{25*len(df)+25}px',
            width=f'{sum(column_widths.values())+100}px',
            margin='0px',
            padding='0px',
        )
    )

    
    grid_dict[name] = grid  # store reference

    scrollable_box = widgets.Box(
        [grid],
        layout=widgets.Layout(
            overflow_x='auto',
            overflow_y='auto',
            width='100%',
            border='1px solid lightgray',
            justify_content='center',
            margin='0px',
            padding='0px',
        )
    )

    sections.append(widgets.HTML(
        f"<h3 style='font-family:Helvetica, Arial, sans-serif; text-align:center; color:#55697d;'>{name}</h3>"
    ))
    sections.append(scrollable_box)

# -----------------------------
# Save Changes Button
# -----------------------------
def save_all_changes(button):
    for name, grid in grid_dict.items():
        # The underlying df in the DataGrid is automatically updated, so we can just fetch it
        tables[name] = grid.data

save_button = widgets.Button(
    description="Save All Changes",
    button_style="success",
    layout=widgets.Layout(width="200px", margin='10px')
)
save_button.on_click(save_all_changes)

sections.append(widgets.HBox([save_button], layout=widgets.Layout(justify_content='center')))

# -----------------------------
# Display the full interface
# -----------------------------
display(widgets.VBox(
    sections, 
    layout=widgets.Layout(
        align_items='center',
        margin='0px',
        padding='0px',
        gap='0px',
    )
))
    

Non-numeric data 'OVER' found for well 'H5'. Converting to NaN.
Non-numeric data 'OVER' found for well 'H7'. Converting to NaN.


Saved as experiment summary file

In [43]:


for k, v in tables.items():
        metadata_dict[k] = v.to_dict()
    
for k, v in data_dict.items():
        metadata_dict[k] = v 

existing_df= pd.read_csv('proposed_summary.csv')
def summarize_tables(tables,target_column='Plate %Change',  metadata_dict= metadata_dict,existing_df=existing_df):
    # Use the first table in the dict to define row/col grid
    first_key = list(tables.keys())[0]
    ref_df = tables[first_key]

    rows = np.repeat(ref_df.index, len(ref_df.columns))
    cols = list(ref_df.columns) * len(ref_df.index) 
    locations = [f"{r}{c}" for r, c in zip(rows, cols)]

    # Step 1: base long DataFrame
    summary = pd.DataFrame({"Location": locations})
    
    for name, df in tables.items():
        summary[name] = df.values.ravel()
        
    
    non_numeric_mask = pd.to_numeric(summary[target_column], errors="coerce").isna()
    non_numeric_df = summary.loc[non_numeric_mask, ["Compound", "Location", target_column]].copy()

    non_numeric_dict = {}
    for compound, group in non_numeric_df.groupby("Compound"):
        value_to_loc = defaultdict(list)  # allows multiple locations per value
        for val, loc in zip(group[target_column], group["Location"]):
            value_to_loc[val].append(loc)  # append instead of overwrite
        non_numeric_dict[compound] = dict(value_to_loc)
        
        
    # Convert all values to numeric, set non-numeric to NaN
    summary[target_column]=summary[target_column].apply(pd.to_numeric, errors="coerce").dropna(axis=0, how="any")
    
    # Define grouping columns
    group_cols = ["Compound", "Concentration", "Cell line"]  # adjust 'Cell line' if your lipid column has a different name

    # Group and compute statistics
    summary_stats = (
        summary
        .groupby(group_cols, dropna=False)
        .agg(
            mean=(target_column, "mean"),
            std=(target_column, "std"),
            count=(target_column, "count"))
        .reset_index()
    )

    
    # Group by Compound and aggregate lists
    summary_stats = summary_stats.groupby("Compound").agg({
        "Concentration": list,
        "mean": list,
        "std": lambda x: [0 if np.isnan(v) else v for v in x],
        "Cell line": "first" 
    }).reset_index()
    
    
    summary_stats["dropped wells"] = summary_stats["Compound"].map(lambda x: str(non_numeric_dict.get(x, {})))
    summary_stats['all data']=summary_stats['all data'] = json.dumps(metadata_dict)
    summary_stats['DMSO_control']=avg_dmso
    summary_stats['positive_control']=avg_pos
    summary_stats['Ratio'] = '2:1'
    summary_stats['uM / nM'] ='uM' 
        # Split by '-' and expand into two column
    split_list = summary_stats['Compound'].str.split('-', n=1)

    # Strip extra whitespace
    summary_stats['Comp1'] = split_list.apply(lambda x: x[0].strip())
    summary_stats['Comp2'] = split_list.apply(lambda x: x[1].strip() if len(x) > 1 else '')
    metadata_splits = metadata_dict['file_path'].split('_')
    summary_stats['Date'] = metadata_splits[0]
    summary_stats['Initials'] = metadata_splits[1]
    summary_stats['Lipid_Type'] = metadata_splits[2]
    summary_stats['Plate_ID'] = metadata_splits[3]
        
    summary_stats = summary_stats[~summary_stats["Compound"].str.contains("control")]
    
    return summary, summary_stats

summary_df, summary_stats = summarize_tables(tables)


# combine metadata with concentration/mean/std lists from metadata_dict
output_rows = []
metadata_cols = summary_stats.columns.to_list()
for idx, row in summary_stats.iterrows():
    
    compound_name = row['Compound']
    # Get lists from metadata_dict
    conc_list = row['Concentration']
    mean_list = row['mean']
    std_list = row['std']
    conc_list = [float(c) for c in conc_list]
    # Combine into tuples, sort by concentration
    combined = sorted(zip(conc_list, mean_list, std_list), key=lambda x: x[0])

    # Unpack back into separate lists
    conc_list, mean_list, std_list = map(list, zip(*combined))    
    # Metadata row (first row)
    metadata_row = [row[col] for col in metadata_cols]
    
    output_rows.append(metadata_row)
    output_rows.append(conc_list)
    output_rows.append(mean_list)
    output_rows.append(std_list)
    output_rows.append([None])

# If new rows were added, merge with existing and save

formatted_df = pd.DataFrame(output_rows)#, columns=metadata_cols)
for col,metadata in zip(formatted_df.columns,metadata_cols):
    formatted_df=formatted_df.rename(columns={col: metadata})

#combined_df = pd.concat([existing_df, formatted_df], ignore_index=True)
formatted_df.to_csv("proposed_summary.csv", index=False)
formatted_df


,Compound,Concentration,mean,std,Cell line,dropped wells,all data,DMSO_control,positive_control,Ratio,uM / nM,Comp1,Comp2,Date,Initials,Lipid_Type,Plate_ID
0,72,"[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[4.025, 5.824999999999999, 11.125, 19.075, 32....","[0.3500000000000001, 0.8381527307120107, 1.680...",POPG,{},"{""plate_layout"": ""Short_Format_Layout_variable...",0.636706,3.881796,2:1,uM,72,,20251001,ch,POPG,plate2.csv
1,2.633745,3.950617,5.925926,8.888889,13.333333,20.0,30.0,NaN,NaN,None,None,None,None,None,None,None,None
2,4.025,5.825,11.125,19.075,32.5,51.2,69.05,NaN,NaN,None,None,None,None,None,None,None,None
3,0.35,0.838153,1.68003,1.114675,2.906315,3.13794,2.330236,NaN,NaN,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None
5,72-73,"[0.234375, 0.46875, 0.9375, 1.875, 3.75, 7.5, 15]","[6.775, 15.075, 31.700000000000003, 36.55, 45....","[0.4272001872658764, 0.9639329160614166, 1.086...",POPG,{},"{""plate_layout"": ""Short_Format_Layout_variable...",0.636706,3.881796,2:1,uM,72,73,20251001,ch,POPG,plate2.csv
6,0.234375,0.46875,0.9375,1.875,3.75,7.5,15.0,NaN,NaN,None,None,None,None,None,None,None,None
7,6.775,15.075,31.7,36.55,45.4,55.225,64.0,NaN,NaN,None,None,None,None,None,None,None,None
8,0.4272,0.963933,1.086278,1.317826,2.22411,0.464579,3.501428,NaN,NaN,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None


Fit and appended to summary of all experiments

In [45]:
df=pd.read_csv('proposed_summary.csv')
def plot_data(df, compound_name, color=["green","blue","red"]):
    counter=0
    # Generation of figure
    plt.figure(figsize=(9, 5))

    # Hide top and right spines
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_linewidth(4)
    plt.gca().spines['bottom'].set_linewidth(4)
# Filter the DataFrame for the given compound
    compound_df = df[df['Comp1'] == compound_name]
    results = []
    x_values_list = []
    fitted_responses = []
    for idx, row in compound_df.iterrows():
        counter+=1
        
        concentration = row['Concentration']  
        response = row['Response']           
        standard_deviation = row.get('STD')
        GI50, n, x_fit, y_fit, auc, r_squared, fitted_response = process_data(concentration, response, standard_deviation)
        results.append((GI50, n, r_squared))
        x_values_list.append(np.linspace(min(concentration), max(concentration), 1000))
        fitted_responses.append(fitted_response)
        
        plt.errorbar(concentration, response, yerr=standard_deviation, fmt='x', capsize=8, markersize=5, color=color[counter], linewidth=3, markeredgewidth=5)
        plt.plot(np.linspace(min(concentration), max(concentration), 1000), fitted_response, color=color[counter], linewidth=3, label=f"n = {row['Rep']}") 
        
    
    # Format the plot
    #plt.xscale('log')
    # Example data
    #x = [0, 5000]
    #y = [50, 50]  # y = 50 across all x values

    #plt.plot(x, y, 'r--', label='y = 50')  # red dashed line
    #plt.ylim(0, 125)
    #plt.xlim(7.5, 2000)
    plt.xlabel('Concentration (µM)',  fontweight='bold')
    plt.title(compound_name,  fontweight='bold')

    # Add a legend
    plt.legend( loc='best')

    plt.show()
# Define a the hill equation logistic function
def logistic(x, GI50, n):
    A = 100
    B = 0
    response = ((A - B) / (1 + (x / GI50)**n)) + B
    return np.minimum(np.maximum(response, 0), 100)

# Function to process each dataset
def process_data(concentration, response, standard_deviation):
    # Fit the logistic curve
    initial_guess = [np.median(concentration), 1]
    params, _ = curve_fit(logistic, concentration, response, p0=initial_guess, sigma=standard_deviation,maxfev=3000000)
    

    GI50, n = params
    predicted = logistic(concentration, GI50, n)

    ss_res = np.sum((response - predicted) ** 2)
    ss_tot = np.sum((response - np.mean(response)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    smooth_x = np.linspace(min(concentration), max(concentration), 1000)
    fitted_response = logistic(smooth_x, GI50, n)
    
    # Integration of GI50 curve using Simpson's Rule
    x_fit = np.linspace(min(concentration), max(concentration), 1000)
    y_fit = logistic(x_fit, GI50, n)

    # Calculation of area under the curve (auc)
    auc = simpson(y=np.asarray(y_fit), x=np.asarray(x_fit))
    

    return GI50, n, x_fit, y_fit, auc, r_squared, fitted_response

def fit_data(df, print_summary=False):
    for idx, row in df.iterrows():
        concentration = row['Concentration']  
        response = row['Response']            
        standard_deviation = row.get('STD') 
        
        GI50, n, x_fit, y_fit, auc, r_squared, fitted_response = process_data(concentration, response, standard_deviation)
        #results.append((GI50, n, r_squared))
        #x_values_list.append(np.linspace(min(concentration), max(concentration), 1000))
        #fitted_responses.append(fitted_response)

        # Print the actual GI50, Hill slope (n), and r_squared
        A, B = 100, 0
        gi50_actual = GI50 * (((A - B) / (50 - B)) - 1) ** (1/n)
        
        if print_summary == True:
            print(f'GI50 = {gi50_actual:.2f}')
            print(f'n = {n:.2f}')
            print(f'R\u00B2 = {r_squared:.4f}')
            print(f'AUC = {auc:.4f}\n')
        
        # Add values to the DataFrame
        df.at[idx, 'GI50_Actual'] = gi50_actual
        df.at[idx, 'Hill_Slope_n'] = n
        df.at[idx, 'R_squared'] = r_squared
        df.at[idx, 'AUC'] = auc
             
    return df

def extract_data(df):
    response_list = []
    concentration_list = []
    std_list=[]
    metadata_keys = df.columns.to_list()
    
    # Identify blank rows
    blank_rows = df.isnull().all(axis=1)

    # Find blocks based on blank rows
    row_blocks = []
    current_block = []
    for idx, is_blank in blank_rows.items():
        if not is_blank:
            current_block.append(idx)
        elif current_block:
            row_blocks.append(current_block)
            current_block = []
    if current_block:
        row_blocks.append(current_block)

    sub_dfs = []
    metadata_list = []

    for block in row_blocks:
        
        block_df = df.loc[block].reset_index(drop=True)
        
        
        # First row is metadata
        metadata = block_df.iloc[0].to_list()
        

        # Second row is header
        header = block_df.iloc[1].tolist()
        

        # Remaining rows are data
        data = block_df.iloc[2:]
        data.columns = header
        data = data.dropna(axis=1, how='all')        

        sub_dfs.append(data)
        metadata_list.append(metadata)
        
        
        response_list.append([float(x) for x in data.loc[2].tolist()])
        concentration_list.append([float(x) for x in data.columns.tolist()])
        std_list.append([float(x) for x in data.loc[3].tolist()])         
        

        
    summary_df = pd.DataFrame(metadata_list, columns=metadata_keys)
    
    #summary_df['Cell death by vehicle control %'] = summary_df['Mean Vehicle Control']/summary_df['Mean Untreated Control']
    
    summary_df['Response'] = response_list
    summary_df['Concentration'] = concentration_list
    summary_df['STD'] = std_list
    summary_df
    if len(std_list[0]) >= 2:
        print(std_list)
        #summary_df=fit_data(summary_df)
    summary_df['Rep'] = summary_df.groupby(summary_df['Comp1'].astype(str) + '_' + summary_df['Comp2'].astype(str)).cumcount() + 1
    
    return sub_dfs, summary_df 

sub_dfs, summary_df  = extract_data(df)  

#plot_data(summary_df, 'Compound 1')

current_summary_df = pd.read_csv('summary_results.csv')

concatenated_summary_df = pd.concat([current_summary_df, summary_df], ignore_index=True)

concatenated_summary_df.to_csv('summary_results.csv',index=False)

concatenated_summary_df

[[0.3500000000000001, 0.8381527307120107, 1.6800297616411433, 1.114674840480398, 2.9063149634316434, 3.137939876203281, 2.330236039546219], [0.4272001872658764, 0.9639329160614166, 1.0862780491200215, 1.3178264933847186, 2.2241103090149714, 0.4645786621588797, 3.50142828000232], [0.15000000000000016, 0.21602468994692878, 0.5567764362830022, 0.9201449161228162, 2.0199009876724157, 3.1538336460039664, 3.639940475703781]]


,Date,Comp1,Comp2,Ratio,uM / nM,DMSO_control,positive_control,dropped wells,all data,Response,Concentration,STD,Rep,Compound,Cell line,Initials,Lipid_Type,Plate_ID,mean,std
0,20251001.0,72.0,NaN,02:01,uM,0.664420,3.320476,{},"{""plate_layout"": ""Short_Format_Layout_variable...","[1.125, 1.7249999999999999, 4.4, 9.975, 16.725...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.2362907813126304, 0.2362907813126304, 0.668...",1,72,POPC,ch,POPC,plate2,NaN,NaN
1,20251001.0,72.0,73.0,02:01,uM,0.664420,3.320476,{},"{""plate_layout"": ""Short_Format_Layout_variable...","[0.47500000000000003, 1.05, 2.7, 8.75, 23.475,...","[0.234375, 0.46875, 0.9375, 1.875, 3.75, 7.5, ...","[0.25, 0.17320508075688776, 0.3464101615137754...",1,72-73,POPC,ch,POPC,plate2,NaN,NaN
2,20251001.0,73.0,NaN,02:01,uM,0.664420,3.320476,{},"{""plate_layout"": ""Short_Format_Layout_variable...","[0.75, 1.475, 3.325, 10.175, 27.725, 55.95, 84...","[0.15625, 0.3125, 0.625, 1.25, 2.5, 5.0, 10.0]","[0.12909944487358058, 0.09574271077563384, 0.2...",1,73,POPC,ch,POPC,plate2,NaN,NaN
3,20251001.0,56.0,NaN,2:1,uM,0.664582,3.321165,{},"{""plate_layout"": ""Short_Format_Layout_Pippette...","[-5.35, -5.6, -5.2, -4.725, -2.875, 1.47499999...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.5196152422706636, 0.7527726527090812, 0.692...",1,56,POPC,ch,POPC,plate1.csv,"[-5.35, -5.6, -5.2, -4.725, -2.875, 1.47499999...","[0.5196152422706636, 0.7527726527090812, 0.692..."
4,20251001.0,56.0,57.0,2:1,uM,0.664582,3.321165,{},"{""plate_layout"": ""Short_Format_Layout_Pippette...","[-5.333333333333333, -6.2, -5.6000000000000005...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.5131601439446881, 0.3999999999999998, 0.608...",1,56-57,POPC,ch,POPC,plate1.csv,"[-5.333333333333333, -6.2, -5.6000000000000005...","[0.5131601439446881, 0.3999999999999998, 0.608..."
5,20251001.0,57.0,NaN,2:1,uM,0.664582,3.321165,{},"{""plate_layout"": ""Short_Format_Layout_Pippette...","[-4.7, -4.9, -4.04, -1.26, 2.16, 9.42, 23.2599...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.48476798574163354, 0.8215838362577491, 1.10...",1,57,POPC,ch,POPC,plate1.csv,"[-4.7, -4.9, -4.04, -1.26, 2.16, 9.42, 23.2599...","[0.48476798574163354, 0.8215838362577491, 1.10..."
6,20251001.0,56.0,NaN,2:1,uM,0.639658,3.909818,{},"{""plate_layout"": ""Short_Format_Layout_Pipette_...","[2.6500000000000004, 3.975, 6.475, 10.75, 18.6...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.05773502691896271, 0.3593976442141305, 0.28...",1,56,POPG,ch,POPG,plate1.csv,"[2.6500000000000004, 3.975, 6.475, 10.75, 18.6...","[0.05773502691896271, 0.3593976442141305, 0.28..."
7,20251001.0,56.0,57.0,2:1,uM,0.639658,3.909818,{},"{""plate_layout"": ""Short_Format_Layout_Pipette_...","[3.6, 5.166666666666667, 9.466666666666667, 19...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.3000000000000001, 0.20816659994661357, 0.64...",1,56-57,POPG,ch,POPG,plate1.csv,"[3.6, 5.166666666666667, 9.466666666666667, 19...","[0.3000000000000001, 0.20816659994661357, 0.64..."
8,20251001.0,57.0,NaN,2:1,uM,0.639658,3.909818,{},"{""plate_layout"": ""Short_Format_Layout_Pipette_...","[4.720000000000001, 6.720000000000001, 13.0400...","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.7949842765740718, 0.8927485648266255, 1.618...",1,57,POPG,ch,POPG,plate1.csv,"[4.720000000000001, 6.720000000000001, 13.0400...","[0.7949842765740718, 0.8927485648266255, 1.618..."
9,20251001.0,72.0,NaN,2:1,uM,0.636706,3.881796,{},"{""plate_layout"": ""Short_Format_Layout_variable...","[4.025, 5.824999999999999, 11.125, 19.075, 32....","[2.633745, 3.950617, 5.925926, 8.888889, 13.33...","[0.3500000000000001, 0.8381527307120107, 1.680...",1,72,POPG,ch,POPG,plate2.csv,"[4.025, 5.824999999999999, 11.125, 19.075, 32....","[0.3500000000000001, 0.8381527307120107, 1.680..."


Replot origin plots in python

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import ast

# Define the two Hill functions
def nlf_Hill1(x, START, END, k, n):
    """Modified Hill function with offset."""
    return START + (END - START) * (x**n / (k**n + x**n))

def nlf_hill(x, Vmax, k, n):
    """Standard Hill function."""
    return Vmax * (x**n / (k**n + x**n))

def parse_data_from_row(row):
    """
    Parse concentration, response, and error data from a dataframe row.
    Assumes data is stored in string format that can be evaluated.
    """
    # Parse the data strings - adjust column names as needed
    concentrations = np.array(ast.literal_eval(row['Concentration']))
    responses = np.array(ast.literal_eval(row['Response']))
    errors = np.array(ast.literal_eval(row['STD']))
    
    return concentrations, responses, errors

def fit_hill_functions(concentrations, responses, errors):
    """
    Fit both Hill function models to the data.
    
    Returns:
    --------
    dict with fit results for both models
    """
    results = {}
    
    # Calculate data characteristics
    response_min = min(responses)
    response_max = max(responses)
    response_range = response_max - response_min
    
    # Fit nlf_Hill1 (Modified Hill with offset)
    # Initial guesses that respect the bounds
    p0_hill1 = [
        response_min,           # START: initial baseline
        100,                    # END: theoretical maximum of 100% efflux
        np.median(concentrations),  # k: median concentration
        1.5                     # n: moderate steepness
    ]
    
    try:
        # More constrained bounds to prevent unrealistic fits
        # END is constrained to be near 100 (theoretical maximum efflux)
        lower_bounds = [
            response_min-5,  # START: allow some flexibility below min
            100,                                    # END: must be close to 100% (theoretical max)
            min(concentrations) * 0.1,             # k: allow below lowest concentration
            0.5                                     # n: minimum meaningful steepness
        ]
        upper_bounds = [
            response_max,  # START: allow some flexibility above max
            101,                                   # END: constrain near 100% with small tolerance
            max(concentrations) * 3,               # k: keep within reasonable extrapolation
            5                                       # n: reasonable maximum steepness
        ]
        
        popt_hill1, pcov_hill1 = curve_fit(
            nlf_Hill1, concentrations, responses, 
            p0=p0_hill1, sigma=errors, absolute_sigma=True,
            bounds=(lower_bounds, upper_bounds),
            maxfev=10000  # Increase max iterations
        )
        
        residuals_hill1 = responses - nlf_Hill1(concentrations, *popt_hill1)
        ss_res_hill1 = np.sum(residuals_hill1**2)
        ss_tot_hill1 = np.sum((responses - np.mean(responses))**2)
        r_squared_hill1 = 1 - (ss_res_hill1 / ss_tot_hill1)
        perr_hill1 = np.sqrt(np.diag(pcov_hill1))
        
        results['hill1'] = {
            'params': popt_hill1,
            'errors': perr_hill1,
            'r_squared': r_squared_hill1,
            'success': True
        }
    except Exception as e:
        results['hill1'] = {'success': False, 'error': str(e)}
    
    # Fit nlf_hill (Standard Hill with shifted data)
    data_shift = min(responses)
    shifted_responses = responses - data_shift
    p0_hill = [max(shifted_responses), np.median(concentrations), 2]
    
    try:
        popt_hill, pcov_hill = curve_fit(
            nlf_hill, concentrations, shifted_responses,
            p0=p0_hill, sigma=errors, absolute_sigma=True,
            bounds=([0.01, 0.01, 0.01], [np.inf, 100, np.inf])
        )
        
        residuals_hill = shifted_responses - nlf_hill(concentrations, *popt_hill)
        ss_res_hill = np.sum(residuals_hill**2)
        ss_tot_hill = np.sum((shifted_responses - np.mean(shifted_responses))**2)
        r_squared_hill = 1 - (ss_res_hill / ss_tot_hill)
        perr_hill = np.sqrt(np.diag(pcov_hill))
        
        results['hill'] = {
            'params': popt_hill,
            'errors': perr_hill,
            'r_squared': r_squared_hill,
            'data_shift': data_shift,
            'success': True
        }
    except Exception as e:
        results['hill'] = {'success': False, 'error': str(e)}
    
    return results

def plot_hill1_fit(concentrations, responses, errors, fit_results, title_info):
    """
    Create plot for nlf_Hill1 (Modified Hill with offset).
    """
    fig, ax = plt.subplots(figsize=(15, 7))
    
    # Plot data
    ax.errorbar(concentrations, responses, yerr=errors, 
                fmt='o', color='blue', markersize=8, capsize=5, capthick=2,
                label='Experimental Data', zorder=3)
    
    if fit_results['hill1']['success']:
        popt = fit_results['hill1']['params']
        perr = fit_results['hill1']['errors']
        r_squared = fit_results['hill1']['r_squared']
        
        # Create smooth curve
        x_smooth = np.linspace(min(concentrations), max(concentrations), 500)
        y_fit = nlf_Hill1(x_smooth, *popt)
        
        ax.plot(x_smooth, y_fit, 'r-', linewidth=2, 
                label=f'nlf_Hill1 Fit (R²={r_squared:.4f})', zorder=2)
        
        # Add horizontal lines
        ax.axhline(y=popt[0], color='green', linestyle='--', alpha=0.5, 
                   label=f'START = {popt[0]:.2f}')
        ax.axhline(y=popt[1], color='orange', linestyle='--', alpha=0.5, 
                   label=f'END = {popt[1]:.2f}')
        
        # Add text box
        textstr = f'START = {popt[0]:.3f} ± {perr[0]:.3f}\n'
        textstr += f'END = {popt[1]:.3f} ± {perr[1]:.3f}\n'
        textstr += f'k (EC50) = {popt[2]:.3f} ± {perr[2]:.3f}\n'
        textstr += f'n = {popt[3]:.3f} ± {perr[3]:.3f}'
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round', 
                facecolor='wheat', alpha=0.8))
    
    # Create title from row info
    title = f"{title_info['Comp1']}-{title_info['Comp2']} | {title_info['Lipid_Type']} | {title_info['Plate_ID']}\nModified Hill Function (nlf_Hill1)"
    
    ax.set_xlabel('Concentration', fontsize=12, fontweight='bold')
    ax.set_ylabel('Response', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def plot_hill_fit(concentrations, responses, errors, fit_results, title_info):
    """
    Create plot for nlf_hill (Standard Hill).
    """
    fig, ax = plt.subplots(figsize=(15, 7))
    
    if fit_results['hill']['success']:
        data_shift = fit_results['hill']['data_shift']
        shifted_responses = responses - data_shift
        
        # Plot shifted data
        ax.errorbar(concentrations, shifted_responses, yerr=errors, 
                    fmt='o', color='blue', markersize=8, capsize=5, capthick=2,
                    label='Shifted Data', zorder=3)
        
        popt = fit_results['hill']['params']
        perr = fit_results['hill']['errors']
        r_squared = fit_results['hill']['r_squared']
        
        # Create smooth curve
        x_smooth = np.linspace(min(concentrations), max(concentrations), 500)
        y_fit = nlf_hill(x_smooth, *popt)
        
        ax.plot(x_smooth, y_fit, 'r-', linewidth=2, 
                label=f'nlf_hill Fit (R²={r_squared:.4f})', zorder=2)
        
        # Add horizontal lines
        ax.axhline(y=popt[0], color='orange', linestyle='--', alpha=0.5, 
                   label=f'Vmax = {popt[0]:.2f}')
        ax.axhline(y=0, color='green', linestyle='--', alpha=0.5, 
                   label='y = 0 (origin)')
        
        # Add text box
        textstr = f'Shift = {data_shift:.3f}\n'
        textstr += f'Vmax = {popt[0]:.3f} ± {perr[0]:.3f}\n'
        textstr += f'k (EC50) = {popt[1]:.3f} ± {perr[1]:.3f}\n'
        textstr += f'n = {popt[2]:.3f} ± {perr[2]:.3f}'
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle='round', 
                facecolor='wheat', alpha=0.8))
    
    # Create title from row info
    title = f"{title_info['Comp1']}-{title_info['Comp2']} | {title_info['Lipid_Type']} | {title_info['Plate_ID']}\nStandard Hill Function (nlf_hill)"
    
    ax.set_xlabel('Concentration', fontsize=12, fontweight='bold')
    ax.set_ylabel('Shifted Response', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def plot_comparison(concentrations, responses, errors, fit_results, title_info):
    """
    Create comparison plot of both fits on original scale.
    """
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Plot data
    ax.errorbar(concentrations, responses, yerr=errors, 
                fmt='o', color='black', markersize=10, capsize=5, capthick=2,
                label='Experimental Data', zorder=4)
    
    x_smooth = np.linspace(min(concentrations), max(concentrations), 500)
    
    # Plot nlf_Hill1 fit
    if fit_results['hill1']['success']:
        popt = fit_results['hill1']['params']
        r_squared = fit_results['hill1']['r_squared']
        y_fit = nlf_Hill1(x_smooth, *popt)
        ax.plot(x_smooth, y_fit, 'r-', linewidth=2.5, 
                label=f'nlf_Hill1 (R²={r_squared:.4f})', zorder=3)
    
    # Plot nlf_hill fit (shifted back to original scale)
    if fit_results['hill']['success']:
        popt = fit_results['hill']['params']
        r_squared = fit_results['hill']['r_squared']
        data_shift = fit_results['hill']['data_shift']
        y_fit = nlf_hill(x_smooth, *popt) + data_shift
        ax.plot(x_smooth, y_fit, 'b--', linewidth=2.5, 
                label=f'nlf_hill + shift (R²={r_squared:.4f})', zorder=2)
    
    # Create title from row info
    title = f"{title_info['Comp1']}-{title_info['Comp2']} | {title_info['Lipid_Type']} | {title_info['Plate_ID']}\nComparison of Hill Function Fits"
    
    ax.set_xlabel('Concentration', fontsize=13, fontweight='bold')
    ax.set_ylabel('Response', fontsize=13, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

def process_dataframe_row(row, plot_type='hill1'):
    """
    Process a single row from the dataframe and create plots.
    
    Parameters:
    -----------
    row : pandas Series
        A row from the dataframe
    plot_type : str
        'hill1', 'hill', 'comparison', or 'all'
    
    Returns:
    --------
    dict with fit results and figure objects
    """
    # Parse data from row
    concentrations, responses, errors = parse_data_from_row(row)
    
    # Create title info dictionary
    title_info = {
        'Comp1': row.get('Comp1', 'Comp1'),
        'Comp2': row.get('Comp2', 'Comp2'),
        'Lipid_Type': row.get('Lipid_Type', 'Lipid'),
        'Plate_ID': row.get('Plate_ID', 'Plate')
    }
    
    # Fit the models
    fit_results = fit_hill_functions(concentrations, responses, errors)
    
    # Generate plots
    figures = {}
    
    if plot_type in ['hill1', 'all']:
        figures['hill1'] = plot_hill1_fit(concentrations, responses, errors, 
                                          fit_results, title_info)
    
    if plot_type in ['hill', 'all']:
        figures['hill'] = plot_hill_fit(concentrations, responses, errors, 
                                        fit_results, title_info)
    
    if plot_type in ['comparison', 'all']:
        figures['comparison'] = plot_comparison(concentrations, responses, errors, 
                                                fit_results, title_info)
    
    return {
        'fit_results': fit_results,
        'figures': figures,
        'title_info': title_info
    }

def create_filename(row):
    """
    Create filename from row data, omitting Comp1 if it's NaN.
    Format: Comp1_Comp2_Lipid_Plate.png or Comp2_Lipid_Plate.png
    """
    comp1 = row.get('Comp1', '')
    comp2 = row.get('Comp2', 'Comp2')
    lipid = row.get('Lipid_Type', 'Lipid')
    plate = row.get('Plate_ID', 'Plate')
    
    # Check if comp1 is NaN or empty
    if pd.isna(comp2) or str(comp2).strip() == '':
        filename = f"20251001_ch_SSA{int(comp1)}_{lipid}.png"
    else:
        filename = f"20251001_ch_SSA{int(comp1)}_SSA_{int(comp2)}_{lipid}.png"
    
    # Clean filename (remove special characters)
    filename = filename.replace('/', '-').replace('\\', '-').replace(' ', '_')
    
    return filename

def create_plot_title(row):
    """
    Create plot title from row data, omitting Comp1 if it's NaN.
    """
    comp1 = row.get('Comp1', '')
    comp2 = row.get('Comp2', 'Comp2')
    lipid = row.get('Lipid_Type', 'Lipid')
    plate = row.get('Plate_ID', 'Plate')
    
    # Check if comp1 is NaN or empty
    if pd.isna(comp2) or str(comp2).strip() == '':
        title = f"{int(comp1)} | {lipid} | {plate}"
    else:
        title = f"{int(comp1)}-{int(comp2)} | {lipid} | {plate}"
    
    return title

def process_and_save_dataframe(df, output_dir='plots', dpi=300):
    """
    Process entire dataframe, fit Hill1 function, save plots, and add fit parameters to dataframe.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Input dataframe with concentration/response data
    output_dir : str
        Directory to save plots
    dpi : int
        Resolution for saved plots
    
    Returns:
    --------
    pandas DataFrame with added columns: GI50, k_Hill, Steepness, R_squared_Hill1
    """
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Make a copy of the dataframe to avoid modifying the original
    df_result = df.copy()
    
    # Initialize new columns with lists
    gi50_list = []
    k_hill_list = []
    steepness_list = []
    r_squared_list = []
    start_list = []
    end_list = []
    
    # Process each row
    for idx, row in df_result.iterrows():
        try:
            # Parse data
            concentrations, responses, errors = parse_data_from_row(row)
            
            # Fit Hill1 model
            fit_results = fit_hill_functions(concentrations, responses, errors)
            
            if fit_results['hill1']['success']:
                params = fit_results['hill1']['params']
                r_squared = fit_results['hill1']['r_squared']
                
                # Store parameters
                # params = [START, END, k, n]
                start_list.append(params[0])
                end_list.append(params[1])
                gi50_list.append(params[2])
                k_hill_list.append(params[2])
                steepness_list.append(params[3])
                r_squared_list.append(r_squared)
                
                # Create custom title
                title = create_plot_title(row)
                title_info = {
                    'Comp1': row.get('Comp1', ''),
                    'Comp2': row.get('Comp2', 'Comp2'),
                    'Lipid_Type': row.get('Lipid_Type', 'Lipid'),
                    'Plate_ID': row.get('Plate_ID', 'Plate'),
                    'custom_title': title
                }
                
                # Create plot
                fig = plot_hill1_fit_custom(concentrations, responses, errors, 
                                           fit_results, title_info)
                
                # Create filename and save
                filename = create_filename(row)
                filepath = os.path.join(output_dir, filename)
                fig.savefig(filepath, dpi=dpi, bbox_inches='tight')
                plt.close(fig)
                
                print(f"Row {idx}: Processed successfully - GI50={params[2]:.4f}, Saved: {filename}")
            else:
                # Append NaN for failed fits
                start_list.append(np.nan)
                end_list.append(np.nan)
                gi50_list.append(np.nan)
                k_hill_list.append(np.nan)
                steepness_list.append(np.nan)
                r_squared_list.append(np.nan)
                print(f"Row {idx}: Fit failed - {fit_results['hill1'].get('error', 'Unknown error')}")
                
        except Exception as e:
            # Append NaN for errors
            start_list.append(np.nan)
            end_list.append(np.nan)
            gi50_list.append(np.nan)
            k_hill_list.append(np.nan)
            steepness_list.append(np.nan)
            r_squared_list.append(np.nan)
            print(f"Row {idx}: Error processing - {str(e)}")
    
    # Add all columns at once
    df_result['Hill1_START'] = start_list
    df_result['Hill1_END'] = end_list
    df_result['GI50'] = gi50_list
    df_result['k_Hill'] = k_hill_list
    df_result['Steepness'] = steepness_list
    df_result['R_squared_Hill1'] = r_squared_list
    
    return df_result

def plot_hill1_fit_custom(concentrations, responses, errors, fit_results, title_info):
    """
    Create plot for nlf_Hill1 with custom title in the style shown.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot data points with error bars (black squares)
    ax.errorbar(concentrations, responses, yerr=errors, 
                fmt='s', color='black', markersize=6, capsize=4, capthick=1.5,
                elinewidth=1.5, label='% influx', zorder=3)
    
    if fit_results['hill1']['success']:
        popt = fit_results['hill1']['params']
        perr = fit_results['hill1']['errors']
        r_squared = fit_results['hill1']['r_squared']
        
        # Create smooth curve
        x_smooth = np.linspace(min(concentrations), max(concentrations), 500)
        y_fit = nlf_Hill1(x_smooth, *popt)
        
        # Plot fit line in red
        ax.plot(x_smooth, y_fit, 'r-', linewidth=2, 
                label='Hill1 Fit of Sheet1', zorder=2)
        
        # Create table with parameters in bottom right
        # Format: parameter name | value ± error
        table_data = [
            ['Parameter', 'Value'],
            ['START', f'{popt[0]:.1f} ± {perr[0]:.1f}'],
            ['END', f'{popt[1]:.1f} ± {perr[1]:.1f}'],
            ['k (EC50)', f'{popt[2]:.1f} ± {perr[2]:.1f}'],
            ['n (Steepness)', f'{popt[3]:.1f} ± {perr[3]:.1f}'],
            ['R²', f'{r_squared:.3f}']
        ]
        
        # Create table
        table = ax.table(cellText=table_data,
                 cellLoc='left',
                 loc='center left', # Align the left side of the table to the bbox coordinates
                 bbox=[1.05, 0.5, 0.5, 0.5], # [x_start, y_start, width, height] in axes coordinates
                 edges='closed')
        
        # Style the table
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1, 1.5)
        
        # Color header row
        for i in range(2):
            cell = table[(0, i)]
            cell.set_facecolor('#E8E8E8')
            cell.set_text_props(weight='bold')
        
        # Alternate row colors
        for i in range(1, 6):
            for j in range(2):
                cell = table[(i, j)]
                if i % 2 == 0:
                    cell.set_facecolor('#F5F5F5')
                else:
                    cell.set_facecolor('white')
    
    # Use custom title
    title = title_info.get('custom_title', 'Hill Function Fit')
    
    ax.set_xlabel('Concentration', fontsize=11)
    ax.set_ylabel('% influx', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold', pad=10)
    
    # Legend in upper left
    ax.legend(loc='upper left', fontsize=9, frameon=True, fancybox=False, 
             edgecolor='black', framealpha=1)
    
    # Set axis limits with some padding
    x_range = max(concentrations) - min(concentrations)
    y_range = max(responses) - min(responses)
    ax.set_xlim(min(concentrations) - 0.1*x_range, max(concentrations) + 0.1*x_range)
    ax.set_ylim(min(responses) - 0.2*y_range, max(responses) + 0.2*y_range)
    
    # Add grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Clean up spines
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['top'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    
    plt.tight_layout()
    return fig

# Example usage:

# Load your dataframe
df = pd.read_csv('summary_results.csv')

# Process all rows, fit Hill1, save plots, and get updated dataframe
df_with_fits = process_and_save_dataframe(df, output_dir='replot_plots', dpi=300)

# Save updated dataframe with fit parameters
#df_with_fits.to_csv('data_with_hill_fits3.csv', index=False)

# The dataframe now has columns: GI50, k_Hill, Steepness, R_squared_Hill1, Hill1_START, Hill1_END
print(df_with_fits[['Comp1', 'Comp2', 'GI50', 'Steepness', 'R_squared_Hill1']].head())


Row 0: Processed successfully - GI50=90.0000, Saved: 20251001_ch_SSA56_POPC.png
Row 1: Processed successfully - GI50=69.8082, Saved: 20251001_ch_SSA56_SSA_57_POPC.png
Row 2: Processed successfully - GI50=48.9541, Saved: 20251001_ch_SSA57_POPC.png
Row 3: Processed successfully - GI50=30.6127, Saved: 20251001_ch_SSA56_POPG.png
Row 4: Processed successfully - GI50=18.3615, Saved: 20251001_ch_SSA56_SSA_57_POPG.png
Row 5: Processed successfully - GI50=16.8721, Saved: 20251001_ch_SSA57_POPG.png
Row 6: Processed successfully - GI50=20.0252, Saved: 20251001_ch_SSA72_POPG.png
Row 7: Processed successfully - GI50=10.9490, Saved: 20251001_ch_SSA72_SSA_73_POPG.png
Row 8: Processed successfully - GI50=11.1827, Saved: 20251001_ch_SSA73_POPG.png
Row 9: Processed successfully - GI50=61.6491, Saved: 20251001_ch_SSA56_POPE-POPG.png
Row 10: Processed successfully - GI50=52.1458, Saved: 20251001_ch_SSA56_SSA_57_POPE-POPG.png
Row 11: Processed successfully - GI50=36.4917, Saved: 20251001_ch_SSA57_POPE-POPG